In [2]:
from utils import first_block_of_day, last_block_of_day, rpc_to_parity
from utils import drop_table, create_action_table
from utils import exeSQL, exeMultipleSQL, insert_action, insert_multiple_actions
from utils import map_list, map_args_group
from tqdm import tqdm_notebook as tqdm

date_start = '2016-10-01'
date_end = '2016-12-31'
block_start = first_block_of_day(int(date_start[:4]), int(date_start[5:7]), int(date_start[8:10]))
block_end = last_block_of_day(int(date_end[:4]), int(date_end[5:7]), int(date_end[8:10]))

table_name = "action_20161001_20161231"

par = 20

In [3]:
print (block_start)
print (block_end)

2357373
2912406


In [4]:
recreate = True
if recreate:
    drop_table('action_20161001_20161231')
    create_action_table('action_20161001_20161231')

In [5]:
def parse_action_create(dict_obj):
    if 'error' in dict_obj:
        return None
    directive = dict_obj['type']
    tx = dict_obj['transactionHash']
    block_num = dict_obj['blockNumber']
    tx_seq = dict_obj['transactionPosition']
    act_seq = 0
    dict_action = dict_obj['action']
    dict_result = dict_obj['result']
    source = dict_action['from']
    target = dict_result['address']
    amount = dict_action['value']
    parsed_entry = [directive, source, target, amount, tx, block_num, tx_seq, act_seq]
    return parsed_entry

In [6]:
def parse_action_call(dict_obj):
    if 'error' in dict_obj:
        return None
    directive = dict_obj['type']
    tx = dict_obj['transactionHash']
    block_num = dict_obj['blockNumber']
    tx_seq = dict_obj['transactionPosition']
    act_seq = 0
    dict_action = dict_obj['action']
    source = dict_action['from']
    target = dict_action['to']
    amount = dict_action['value']
    parsed_entry = [directive, source, target, amount, tx, block_num, tx_seq, act_seq]
    return parsed_entry

In [7]:
def parse_action_reward(dict_obj):
    dict_action = dict_obj['action']
    directive = dict_obj['type'] + "-" + dict_action['rewardType']
    tx = dict_obj['transactionHash']
    block_num = dict_obj['blockNumber']
    tx_seq = -1
    act_seq = 0
    source = 'None'
    target = dict_action['author']
    amount = dict_action['value']
    parsed_entry = [directive, source, target, amount, tx, block_num, tx_seq, act_seq]
    return parsed_entry

In [8]:
def fetch_entries_from_actions(actions):
    parsed_entries = []
    for act in actions:
        if act['type'] == 'call':
            parsed_entry = parse_action_call(act)
        elif act['type'] == 'create':
            parsed_entry = parse_action_create(act)
        elif act['type'] == 'reward':
            parsed_entry = parse_action_reward(act)
        else:
            print(act)
        if parsed_entry != None:
            parsed_entries.append(parsed_entry)
    last_tx_hash = ''
    act_seq = -1
    for en in parsed_entries:
        if en[4] != last_tx_hash:
            last_tx_hash=  en[4]
            act_seq = 0
        else:
            act_seq += 1
        en[7] = act_seq
    return parsed_entries

In [9]:
def delete_error_actions(actions):
    err_txs_list = []
    for act in actions:
        if 'error' in act:
            err_txs_list.append(act['transactionHash'])
    tailored_actions_list = []
    for act in actions:
        if not act['transactionHash'] in err_txs_list:
            tailored_actions_list.append(act)
    return tailored_actions_list

In [12]:
def parse_blocks(block_start, block_end, table_name):
#     print (block_start, block_end, table_name)
#     return
    for bn in tqdm(range(block_start, block_end)):
#         print ("######### Start to process block {} #########".format(bn))
        method = 'trace_block'
        params = [hex(bn)]
        actions_per_block = rpc_to_parity(method, params).json()['result']
#         print (actions_per_block)
        tailored_actions = delete_error_actions(actions_per_block)
        parsed_entries = fetch_entries_from_actions(tailored_actions)
        insert_multiple_actions(parsed_entries, table_name = table_name)

In [13]:
parse_blocks(block_start, 2357973, 'action_tmp')

In [11]:
interval_start = block_start
flag = True

while(flag):
    if interval_start + 600 >= block_end + 1:
        interval_end = block_end + 1
        flag = False
    else:
        interval_end = interval_start + 600
    print ("Blocks from {} to {} are in process!".format(interval_start, interval_end))
    args_groups_list = []
    sub_range_len = (interval_end - interval_start) // par
    for i in range(par-1):
        tmp_group = [i*sub_range_len+interval_start, (i+1)*sub_range_len+interval_start, table_name]
        args_groups_list.append(tmp_group)
    tmp_group = [(par-1)*sub_range_len+interval_start, interval_end, table_name]
    args_groups_list.append(tmp_group)
    print (map_args_group(args_groups_list, parse_blocks))
    interval_start += 600

# parse_blocks(block_start, block_end, table_name)

Blocks from 2357373 to 2357973 are in process!
2357403 2357433 action_20161001_20161231
2357523 2357553 action_20161001_20161231
2357433 2357463 action_20161001_20161231
2357493 2357523 action_20161001_20161231
2357373 2357403 action_20161001_20161231
2357583 2357613 action_20161001_20161231
2357763 2357793 action_20161001_20161231
2357793 2357823 action_20161001_20161231
2357463 2357493 action_20161001_20161231
2357883 2357913 action_20161001_20161231
2357823 2357853 action_20161001_20161231
2357703 2357733 action_20161001_20161231
2357733 2357763 action_20161001_20161231
2357853 2357883 action_20161001_20161231
2357943 2357973 action_20161001_20161231
2357643 2357673 action_20161001_20161231
2357673 2357703 action_20161001_20161231
2357613 2357643 action_20161001_20161231
2357553 2357583 action_20161001_20161231
2357913 2357943 action_20161001_20161231
[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]
Blocks from 

2362353 2362383 action_20161001_20161231
2362323 2362353 action_20161001_20161231
2362233 2362263 action_20161001_20161231
2362443 2362473 action_20161001_20161231
2362743 2362773 action_20161001_20161231
2362473 2362503 action_20161001_20161231
2362653 2362683 action_20161001_20161231
2362683 2362713 action_20161001_20161231
2362713 2362743 action_20161001_20161231
2362533 2362563 action_20161001_20161231
2362563 2362593 action_20161001_20161231
2362503 2362533 action_20161001_20161231
2362623 2362653 action_20161001_20161231
2362593 2362623 action_20161001_20161231
[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]
Blocks from 2362773 to 2363373 are in process!
2362863 2362893 action_20161001_20161231
2362833 2362863 action_20161001_20161231
2363073 2363103 action_20161001_20161231
2362983 2363013 action_20161001_20161231
2363013 2363043 action_20161001_20161231
2362953 2362983 action_20161001_20161231
2363193 2363

2367423 2367453 action_20161001_20161231
2367393 2367423 action_20161001_20161231
2367273 2367303 action_20161001_20161231
2367363 2367393 action_20161001_20161231
2367543 2367573 action_20161001_20161231
2367213 2367243 action_20161001_20161231
[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]
Blocks from 2367573 to 2368173 are in process!
2367573 2367603 action_20161001_20161231
2367663 2367693 action_20161001_20161231
2367693 2367723 action_20161001_20161231
2367633 2367663 action_20161001_20161231
2367603 2367633 action_20161001_20161231
2367813 2367843 action_20161001_20161231
2367783 2367813 action_20161001_20161231
2367873 2367903 action_20161001_20161231
2367903 2367933 action_20161001_20161231
2367933 2367963 action_20161001_20161231
2367753 2367783 action_20161001_20161231
2367843 2367873 action_20161001_20161231
2368023 2368053 action_20161001_20161231
2367993 2368023 action_20161001_20161231
2367963 2367

2372073 2372103 action_20161001_20161231
2372313 2372343 action_20161001_20161231
2372343 2372373 action_20161001_20161231
2372463 2372493 action_20161001_20161231
2372493 2372523 action_20161001_20161231
2372523 2372553 action_20161001_20161231
2372373 2372403 action_20161001_20161231
2372403 2372433 action_20161001_20161231
2372553 2372583 action_20161001_20161231
2372733 2372763 action_20161001_20161231
2372673 2372703 action_20161001_20161231
2372643 2372673 action_20161001_20161231
2372943 2372973 action_20161001_20161231
2372823 2372853 action_20161001_20161231
2372913 2372943 action_20161001_20161231
2372583 2372613 action_20161001_20161231
2372433 2372463 action_20161001_20161231
2372883 2372913 action_20161001_20161231
2372703 2372733 action_20161001_20161231
2372763 2372793 action_20161001_20161231
2372793 2372823 action_20161001_20161231
2372613 2372643 action_20161001_20161231
2372853 2372883 action_20161001_20161231
[None, None, None, None, None, None, None, None, None, No

2377203 2377233 action_20161001_20161231
2377233 2377263 action_20161001_20161231
2377383 2377413 action_20161001_20161231
2377443 2377473 action_20161001_20161231
2377533 2377563 action_20161001_20161231
2377623 2377653 action_20161001_20161231
2377593 2377623 action_20161001_20161231
2377653 2377683 action_20161001_20161231
2377683 2377713 action_20161001_20161231
2377473 2377503 action_20161001_20161231
2377743 2377773 action_20161001_20161231
2377563 2377593 action_20161001_20161231
2377713 2377743 action_20161001_20161231
2377503 2377533 action_20161001_20161231
2377413 2377443 action_20161001_20161231
[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]
Blocks from 2377773 to 2378373 are in process!
2377773 2377803 action_20161001_20161231
2377923 2377953 action_20161001_20161231
2378043 2378073 action_20161001_20161231
2377803 2377833 action_20161001_20161231
2377953 2377983 action_20161001_20161231
2378013 2378

...

2892483 2892513 action_20161001_20161231
2892363 2892393 action_20161001_20161231
2892183 2892213 action_20161001_20161231
2892513 2892543 action_20161001_20161231
2892423 2892453 action_20161001_20161231
[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]
Blocks from 2892573 to 2893173 are in process!
2892633 2892663 action_20161001_20161231
2892753 2892783 action_20161001_20161231
2892783 2892813 action_20161001_20161231
2892723 2892753 action_20161001_20161231
2892813 2892843 action_20161001_20161231
2892933 2892963 action_20161001_20161231
2892843 2892873 action_20161001_20161231
2892573 2892603 action_20161001_20161231
2893023 2893053 action_20161001_20161231
2892603 2892633 action_20161001_20161231
2892693 2892723 action_20161001_20161231
2893143 2893173 action_20161001_20161231
2892993 2893023 action_20161001_20161231
2892963 2892993 action_20161001_20161231
2893053 2893083 action_20161001_20161231
2892903 2892

2897463 2897493 action_20161001_20161231
2897643 2897673 action_20161001_20161231
2897553 2897583 action_20161001_20161231
2897673 2897703 action_20161001_20161231
2897493 2897523 action_20161001_20161231
2897433 2897463 action_20161001_20161231
2897403 2897433 action_20161001_20161231
2897523 2897553 action_20161001_20161231
2897373 2897403 action_20161001_20161231
2897733 2897763 action_20161001_20161231
2897763 2897793 action_20161001_20161231
2897793 2897823 action_20161001_20161231
2897823 2897853 action_20161001_20161231
2897703 2897733 action_20161001_20161231
2897853 2897883 action_20161001_20161231
2897613 2897643 action_20161001_20161231
2897913 2897943 action_20161001_20161231
2897943 2897973 action_20161001_20161231
2897883 2897913 action_20161001_20161231
2897583 2897613 action_20161001_20161231
[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]
Blocks from 2897973 to 2898573 are in process!
2898063 2898

2902293 2902323 action_20161001_20161231
2902533 2902563 action_20161001_20161231
2902623 2902653 action_20161001_20161231
2902233 2902263 action_20161001_20161231
2902653 2902683 action_20161001_20161231
2902713 2902743 action_20161001_20161231
2902593 2902623 action_20161001_20161231
2902353 2902383 action_20161001_20161231
2902743 2902773 action_20161001_20161231
2902503 2902533 action_20161001_20161231
2902683 2902713 action_20161001_20161231
2902563 2902593 action_20161001_20161231
[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]
Blocks from 2902773 to 2903373 are in process!
2902773 2902803 action_20161001_20161231
2902923 2902953 action_20161001_20161231
2902953 2902983 action_20161001_20161231
2902803 2902833 action_20161001_20161231
2902833 2902863 action_20161001_20161231
2902893 2902923 action_20161001_20161231
2903013 2903043 action_20161001_20161231
2902863 2902893 action_20161001_20161231
2903253 2903

2907513 2907543 action_20161001_20161231
2907423 2907453 action_20161001_20161231
2907543 2907573 action_20161001_20161231
2907183 2907213 action_20161001_20161231
[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]
Blocks from 2907573 to 2908173 are in process!
2907573 2907603 action_20161001_20161231
2907633 2907663 action_20161001_20161231
2907663 2907693 action_20161001_20161231
2907603 2907633 action_20161001_20161231
2907693 2907723 action_20161001_20161231
2907813 2907843 action_20161001_20161231
2907723 2907753 action_20161001_20161231
2907933 2907963 action_20161001_20161231
2908053 2908083 action_20161001_20161231
2907783 2907813 action_20161001_20161231
2908023 2908053 action_20161001_20161231
2907753 2907783 action_20161001_20161231
2908113 2908143 action_20161001_20161231
2907873 2907903 action_20161001_20161231
2907903 2907933 action_20161001_20161231
2907843 2907873 action_20161001_20161231
2907993 2908

2912373 2912374 action_20161001_20161231
2912375 2912376 action_20161001_20161231
2912378 2912379 action_20161001_20161231
2912376 2912377 action_20161001_20161231
2912377 2912378 action_20161001_20161231
2912374 2912375 action_20161001_20161231
2912380 2912381 action_20161001_20161231
2912379 2912380 action_20161001_20161231
2912385 2912386 action_20161001_20161231
[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]
2912381 2912382 action_20161001_20161231
2912384 2912385 action_20161001_20161231
2912388 2912389 action_20161001_20161231
2912386 2912387 action_20161001_20161231
2912387 2912388 action_20161001_20161231
2912382 2912383 action_20161001_20161231
2912383 2912384 action_20161001_20161231
2912390 2912391 action_20161001_20161231
2912391 2912392 action_20161001_20161231
2912392 2912407 action_20161001_20161231
2912389 2912390 action_20161001_20161231


In [142]:
method = 'trace_block'
params = [hex(2357441)]
resp = rpc_to_parity(method, params)
print (resp.json()['result'])

[{'blockHash': '0x10c10eda256b5956af5651d7758575396cedbc97de6250221854ad6e8ff4e18a', 'traceAddress': [], 'blockNumber': 2357441, 'transactionPosition': 0, 'subtraces': 0, 'type': 'create', 'action': {'init': '0x6060604052600261010860005055604051611b51380380611b51833981016040528080518201919060200180519060200190919080519060200190919050505b805b83835b600060018351016001600050819055503373ffffffffffffffffffffffffffffffffffffffff16600260005060016101008110156100025790900160005b5081905550600161010260005060003373ffffffffffffffffffffffffffffffffffffffff16815260200190815260200160002060005081905550600090505b825181101561016e5782818151811015610002579060200190602002015173ffffffffffffffffffffffffffffffffffffffff166002600050826002016101008110156100025790900160005b508190555080600201610102600050600085848151811015610002579060200190602002015173ffffffffffffffffffffffffffffffffffffffff168152602001908152602001600020600050819055505b80600101905080506100c2565b816000600050819055505b505050806101056000508190555061018

In [ ]:
method = 'trace_transaction'
params = ["0x7b576217ce86740c92af9cd6b8d40ffdc7323df8aa19c1cb33560ee90cc6c421"]
resp_trace_tx = rpc_to_parity(method, params)
print (resp_trace_tx.json()['result'])

In [167]:
method = 'eth_getBlockByNumber'
params = [hex(2357374), True]
resp = rpc_to_parity(method, params)
print (resp.json()['result'])

{'hash': '0x2ab8efe7da15fce005a385e39ab3a7ee0dc2b8e639de66b58f0db599212d66d3', 'number': '0x23f87e', 'parentHash': '0xd074e73c4dca9e1769c5f1bc016b23a9c0f5d96ab44a453d369319c89d9b6064', 'author': '0xf3b9d2c81f2b24b0fa0acaaa865b7d9ced5fc2fb', 'gasUsed': '0x14820', 'timestamp': '0x57eefc85', 'size': '0x3de', 'stateRoot': '0x4fc1793f8af71dfaec37fffffcd05c1b75df585fdd4a7b7eb81ccf7c6bd2e374', 'totalDifficulty': '0x3ae899048133fd3e1', 'mixHash': '0xa690321199f9175d001863ad0e78fddb03cfc9e5202762957402044c48244acd', 'gasLimit': '0x16e358', 'logsBloom': '0x000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [13]:
method = 'trace_replayTransaction'
params = ["0x77656b0b74ea1f8b158067c8371617aae7ab06f05c6c13001606a763688602c3", ['trace']]
resp_trace_replay_tx = rpc_to_parity(method, params)
print (resp_trace_replay_tx.json()['result'])

{'output': '0x0000000000000000000000000000000000000000000000000000000000000001', 'stateDiff': None, 'trace': [{'subtraces': 0, 'type': 'call', 'action': {'input': '0xa9059cbb0000000000000000000000000328307a6cab168e3947bfaec99bf5b64573383f00000000000000000000000000000000000000000000000c9a80665ea1ae5000', 'gas': '0x1eed8', 'from': '0xd9bd20efca7b0e6606b969548b1516c08d37374b', 'value': '0x0', 'to': '0xaf30d2a7e90d7dc361c8c4585e9bb7d2f6f15bc7', 'callType': 'call'}, 'traceAddress': [], 'result': {'output': '0x0000000000000000000000000000000000000000000000000000000000000001', 'gasUsed': '0x7310'}}], 'vmTrace': None}


In [22]:
import json

resp_string = str(resp.json())
resp_string = resp_string.replace("\'", "\"")
resp_string = resp_string.replace("None", "\"None\"")
resp_data = json.loads(resp_string)
print (resp_data)

{'result': [{'blockHash': '0x7eb25504e4c202cf3d62fd585d3e238f592c780cca82dacb2ed3cb5b38883add', 'traceAddress': [], 'blockNumber': 3068185, 'action': {'input': '0x', 'gas': '0x0', 'from': '0xaa7b131dc60b80d3cf5e59b5a21a666aa039c951', 'value': '0x4768d7effc3fbe', 'to': '0xd40aba8166a212d6892125f079c33e6f5ca19814', 'callType': 'call'}, 'transactionPosition': 0, 'subtraces': 0, 'type': 'call', 'result': {'output': '0x', 'gasUsed': '0x0'}, 'transactionHash': '0x07da28d752aba3b9dd7060005e554719c6205c8a3aea358599fc9b245c52f1f6'}, {'blockHash': '0x7eb25504e4c202cf3d62fd585d3e238f592c780cca82dacb2ed3cb5b38883add', 'traceAddress': [], 'blockNumber': 3068185, 'action': {'input': '0x', 'gas': '0x0', 'from': '0x4f11ba23bb526c0486d83c6a8f18f632f3fc172a', 'value': '0x446cde325fbfbe', 'to': '0x7ed1e469fcb3ee19c0366d829e291451be638e59', 'callType': 'call'}, 'transactionPosition': 1, 'subtraces': 0, 'type': 'call', 'result': {'output': '0x', 'gasUsed': '0x0'}, 'transactionHash': '0x056f11efb5da4ff7cf85